In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
import numpy as np
from export.saver import read_pickle
from helpers.layout import Layout
from helpers.graph_helpers import *

from placement.arrange import *
from placement.place import *
from copy import copy, deepcopy
from icecream import ic
from visuals.plotter import plot_general
from itertools import product, combinations
from fixes.reporter import Reporter
from new_solutions.selection import Cook
from domains.range import Range
from new_solutions.interfaces import ResultsLog
from fixes.problem_types.overlap_id import *
from read.svg_reader import SVGReader
from adjacencies.adjacency import AdjacencyGenerator

In [3]:
case  = "amber_a_f01.svg"
sv = SVGReader(case)
sv.run()
ag = AdjacencyGenerator(sv.layout, buffer_size=0.17)
ag.run() # TODO shouldnt run if sv.layout is empty
# ag.draw_graph()

In [4]:
G = ag.layout.graph
domains = ag.layout.domains
arr = initialize_arr(ag.layout)
graph_domains = GraphDomains(G, ag.layout.domains)
arr

array([['bedroom_1', 'laundry', 'm_bath', 'm_bedroom'],
       ['bedroom_2', '', '', '']], dtype=object)

In [5]:
arr1 = place_next_row(graph_domains, arr)
arr1

arrange> nodes with north AND east relationships for members of row 1: ['m_bedroom']


array([['bedroom_1', 'laundry', 'm_bath', 'm_bedroom'],
       ['bedroom_2', 'transit_space', 'bath', 'm_bedroom']], dtype=object)

In [6]:
arr2  = place_next_row(graph_domains, arr1)
arr2

array([['bedroom_1', 'laundry', 'm_bath', 'm_bedroom'],
       ['bedroom_2', 'transit_space', 'bath', 'm_bedroom'],
       ['bedroom_2', 'kitchen', 'dining', 'm_closet']], dtype=object)

In [8]:
arr3  = place_next_row(graph_domains, arr2)
arr3

array([['bedroom_1', 'laundry', 'm_bath', 'm_bedroom'],
       ['bedroom_2', 'transit_space', 'bath', 'm_bedroom'],
       ['bedroom_2', 'kitchen', 'dining', 'm_closet'],
       ['bedroom_2', '', '', '']], dtype=object)

In [26]:
unplaced = list(get_unplaced(arr3, graph_domains.domains))
unplaced

['closet']

In [27]:
arr3[-1, -(len(unplaced)):] = unplaced
arr3

array([['bedroom_1', 'laundry', 'm_bath', 'm_bedroom'],
       ['bedroom_2', 'transit_space', 'bath', 'm_bedroom'],
       ['bedroom_2', 'kitchen', 'dining', 'm_closet'],
       ['bedroom_2', '', '', 'closet']], dtype=object)

In [35]:
# find indices that are missing 
row = copy(arr3[-1])
indices = [ix for ix, item in enumerate(row) if not item]
row[indices] = arr3[-2, indices]
row



array(['bedroom_2', 'kitchen', 'dining', 'closet'], dtype=object)

In [34]:
arr3[-2, [1,2]]

array(['kitchen', 'dining'], dtype=object)

In [9]:
arr4 = handle_unplaced(graph_domains, arr3)
arr4

array([['bedroom_1', 'laundry', 'm_bath', 'm_bedroom'],
       ['bedroom_2', 'transit_space', 'bath', 'm_bedroom'],
       ['bedroom_2', 'kitchen', 'dining', 'm_closet'],
       ['bedroom_2', 'kitchen', 'dining', 'closet']], dtype=object)

In [10]:
darr = create_domains_arr(arr4, domains)
darr

array([[Domain(bedroom_1, x=[0.00-3.22], y=[3.29-6.09]),
        Domain(laundry, x=[3.38-4.98], y=[4.31-6.06]),
        Domain(m_bath, x=[5.30-7.62], y=[4.79-6.09]),
        Domain(m_bedroom, x=[7.80-9.93], y=[3.35-6.09])],
       [Domain(bedroom_2, x=[0.00-3.22], y=[0.00-3.15]),
        Domain(transit_space, x=[3.36-5.16], y=[2.41-4.19]),
        Domain(bath, x=[5.30-7.62], y=[3.35-4.58]),
        Domain(m_bedroom, x=[7.80-9.93], y=[3.35-6.09])],
       [Domain(bedroom_2, x=[0.00-3.22], y=[0.00-3.15]),
        Domain(kitchen, x=[3.36-5.16], y=[0.00-2.24]),
        Domain(dining, x=[5.42-8.78], y=[0.00-3.15]),
        Domain(m_closet, x=[8.93-9.93], y=[2.69-3.29])],
       [Domain(bedroom_2, x=[0.00-3.22], y=[0.00-3.15]),
        Domain(kitchen, x=[3.36-5.16], y=[0.00-2.24]),
        Domain(dining, x=[5.42-8.78], y=[0.00-3.15]),
        Domain(closet, x=[8.93-9.93], y=[-0.05-2.36])]], dtype=object)

In [11]:
darr1 = place_nodes(darr)
darr1

array([[Domain(bedroom_1, x=[0.00-3.22], y=[-2.80-0.00]),
        Domain(laundry, x=[3.22-4.82], y=[-1.75-0.00]),
        Domain(m_bath, x=[4.82-7.14], y=[-1.30-0.00]),
        Domain(m_bedroom, x=[7.14-9.27], y=[-2.74-0.00])],
       [Domain(bedroom_2, x=[0.00-3.22], y=[-5.95--2.80]),
        Domain(transit_space, x=[3.22-5.02], y=[-3.53--1.75]),
        Domain(bath, x=[5.02-7.34], y=[-2.53--1.30]),
        Domain(m_bedroom, x=[7.14-9.27], y=[-2.74-0.00])],
       [Domain(bedroom_2, x=[0.00-3.22], y=[-5.95--2.80]),
        Domain(kitchen, x=[3.22-5.02], y=[-5.77--3.53]),
        Domain(dining, x=[5.02-8.38], y=[-5.68--2.53]),
        Domain(m_closet, x=[8.38-9.38], y=[-3.34--2.74])],
       [Domain(bedroom_2, x=[0.00-3.22], y=[-5.95--2.80]),
        Domain(kitchen, x=[3.22-5.02], y=[-5.77--3.53]),
        Domain(dining, x=[5.02-8.38], y=[-5.68--2.53]),
        Domain(closet, x=[8.38-9.38], y=[-5.75--3.34])]], dtype=object)

In [12]:
plot_general(create_new_domains(darr1))

In [13]:
playout = create_placement_and_update_layout(ag.layout)
re = Reporter(playout)
re.run()
re.problems

arrange> nodes with north AND east relationships for members of row 1: ['m_bedroom']
iterations exceeded, handling unplaced.. 


[Problem(ix=1, HOLE, ['transit_space', 'laundry', 'm_bath', 'bath'], resolved=False,  n_actions=4,
 Problem(ix=2, OVERLAP, ['m_bedroom', 'bath'], resolved=False,  n_actions=2,
 Problem(ix=3, OVERLAP, ['m_bedroom', 'dining'], resolved=False,  n_actions=4]

In [14]:
c = Cook(re)

initializing.. 0
running again -> 1
problem being studied: -> Problem(ix=2, OVERLAP, ['m_bedroom', 'bath'], resolved=False,  n_actions=2
next best layout bath-SQUEEZE-for-OVERLAP near ['m_bedroom', 'bath']2-PS:0.60-S:1.30


In [15]:
c.run_again()

running again -> 2
problem being studied: -> Problem(ix=1, HOLE, ['transit_space', 'laundry', 'm_bath', 'bath'], resolved=False,  n_actions=4
next best layout bath-PULL-for-HOLE near ['transit_space', 'laundry']2-PS:0.51-S:1.25


In [16]:
c.run_again()

running again -> 3
problem being studied: -> Problem(ix=3, OVERLAP, ['bath', 'transit_space'], resolved=False,  n_actions=4
skipping bc prev domains are in history
next best layout m_bedroom-PUSH-for-OVERLAP near ['m_bedroom', 'dining']2-PS:0.60-S:1.30


In [17]:
c.run_again()

running again -> 4
problem being studied: -> Problem(ix=3, SIDE_HOLE, ['m_closet', 'm_bedroom'], resolved=False,  n_actions=2
next best layout m_closet-STRETCH-for-SIDE_HOLE near ['m_closet', 'm_bedroom']1-PS:0.34-S:0.67


In [18]:
c.run_again()

running again -> 5
problem being studied: -> Problem(ix=3, OVERLAP, ['bath', 'transit_space'], resolved=False,  n_actions=4
next best layout bath-SQUEEZE-for-OVERLAP near ['bath', 'transit_space']1-PS:0.25-S:0.62


In [19]:
c.run_again()

running again -> 6
problem being studied: -> Problem(ix=2, HOLE, ['transit_space', 'laundry', 'm_bath', 'bath'], resolved=False,  n_actions=4
skipping bc prev domains are in history
next best layout laundry-STRETCH-for-HOLE near ['transit_space', 'laundry']1-PS:0.35-S:0.68


In [20]:
c.run_again()

running again -> 7
problem being studied: -> Problem(ix=2, OVERLAP, ['m_bath', 'laundry'], resolved=False,  n_actions=2
next best layout m_bath-SQUEEZE-for-OVERLAP near ['m_bath', 'laundry']0-PS:0.26-S:0.13


In [21]:
plot_general(c.bl_hist[-1].layout.domains)